In [1]:
import os
os.environ["PYSPARK_PYTHON"] = r"C:\Users\DELL\AppData\Local\Programs\Python\Python310\python.exe"
os.environ["PYSPARK_DRIVER_PYTHON"] = r"C:\Users\DELL\AppData\Local\Programs\Python\Python310\python.exe"

In [2]:
# %pip install pyspark

# Khởi tạo Spark Session để đọc file csv

In [3]:
from pyspark.sql import SparkSession, functions as F

# Tạo SparkSession
spark_session = (
    SparkSession
    .builder
    .appName("E-Commerce-Fecom")
    .getOrCreate()
)

if spark_session:
    print("SparkSession created successfully.")
else:
    print("Error: Failed to create SparkSession.")


SparkSession created successfully.


## Câu 1: Hãy đọc dữ liệu từ các file csv, sử dụng tự suy ra kiểu dữ liệu cho mỗi cột. 

#### Sử dụng hàm read để đọc các file csv. Bởi vì giữa các cột được ngăn cách bằng dấu ";" cho nên ta dùng thêm option separate để chỉ định kí tự phân tách cột. Sử dụng inferSchema để tự nội suy kiểu dữ liệu.

In [16]:
items_df = (spark_session.read
            .option("header", True)
            .option("inferSchema",True)
            .option("sep", ";")
            .csv("resource/Order_items.csv"))
reviews_df = (spark_session.read
              .option("header", True)
              .option("inferSchema",True)
              .option("sep", ";")
              .csv("resource/Order_Reviews.csv"))
orders_df = (spark_session.read
              .option("header", True)
              .option("inferSchema",True)
              .option("sep", ";")
              .csv("resource/Orders.csv"))
products_df = (spark_session.read
              .option("header", True)
              .option("inferSchema",True)
              .option("sep", ";")
              .csv("resource/Products.csv"))
customers_df = (spark_session.read
              .option("header", True)
              .option("inferSchema",True)
              .option("sep", ";")
              .csv("resource/Customer_List.csv"))

#### Xem schema của từng file.

In [17]:
for name, df in [("orders_df", orders_df), ("customers_df", customers_df), ("items_df", items_df), ("products_df", products_df), ("reviews_df", reviews_df)]:
    print("\n", name)
    df.printSchema()


 orders_df
root
 |-- Order_ID: string (nullable = true)
 |-- Customer_Trx_ID: string (nullable = true)
 |-- Order_Status: string (nullable = true)
 |-- Order_Purchase_Timestamp: timestamp (nullable = true)
 |-- Order_Approved_At: timestamp (nullable = true)
 |-- Order_Delivered_Carrier_Date: timestamp (nullable = true)
 |-- Order_Delivered_Customer_Date: timestamp (nullable = true)
 |-- Order_Estimated_Delivery_Date: timestamp (nullable = true)


 customers_df
root
 |-- Customer_Trx_ID: string (nullable = true)
 |-- Subscriber_ID: string (nullable = true)
 |-- Subscribe_Date: date (nullable = true)
 |-- First_Order_Date: date (nullable = true)
 |-- Customer_Postal_Code: string (nullable = true)
 |-- Customer_City: string (nullable = true)
 |-- Customer_Country: string (nullable = true)
 |-- Customer_Country_Code: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)


 items_df
root
 |-- Order_ID: string (nullable = true)
 |-- Order_Item_ID

#### Xem dữ liệu 5 dòng đầu  của từng file

In [18]:
for name, df in [("orders_df", orders_df), ("customers_df", customers_df), ("items_df", items_df), ("products_df", products_df), ("reviews_df", reviews_df)]:
    print("\n", name)
    df.show(5)


 orders_df
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            Order_ID|     Customer_Trx_ID|Order_Status|Order_Purchase_Timestamp|  Order_Approved_At|Order_Delivered_Carrier_Date|Order_Delivered_Customer_Date|Order_Estimated_Delivery_Date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2023-10-02 10:56:00|2023-10-02 11:07:00|         2023-10-04 19:55:00|          2023-10-10 21:25:00|          2023-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2024-07-24 20:41:00|2024-07-26 03:24:00|         2024-07-26 14:31:00|          2024-08-07 15:27:00|          2024-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b

In [19]:
print(items_df.columns)

['Order_ID', 'Order_Item_ID', 'Product_ID', 'Seller_ID', 'Shipping_Limit_Date', 'Price', 'Freight_Value']


## Câu 2: Thống kê tổng số đơn hàng, số lượng khách hàng và người bán

In [20]:
# Dùng select distinct để lấy danh sách cần tính và đếm số lượng
total_orders = orders_df.select("Order_ID").distinct().count()
total_customers = customers_df.select("Customer_Trx_ID").distinct().count()
total_sellers = items_df.select("Seller_ID").distinct().count()
print(f"Total orders: {total_orders}, Total customers: {total_customers}, Total sellers: {total_sellers}")

Total orders: 99441, Total customers: 99442, Total sellers: 3095


## Câu 3: Phân tích số lượng đơn hàng theo quốc gia, sắp xếp theo thứ tự giảm dần

In [21]:
orders_country = (
    # join 2 bảng là orders_df và customers_df với key Customer_Trx_ID
    orders_df
    .join(customers_df.select("Customer_Country", "Customer_Trx_ID"), how="inner", on="Customer_Trx_ID")
    # group theo Customer_Country
    .groupBy("Customer_Country")
    # Đếm số lượng orders
    .agg(F.count_distinct("Order_ID").alias("Total_Orders"))
    # Sắp xếp theo số lượng orders giảm dần
    .orderBy(F.col("Total_Orders").desc())
)
orders_country.show(50, truncate=False)


+----------------+------------+
|Customer_Country|Total_Orders|
+----------------+------------+
|Germany         |41754       |
|France          |12848       |
|Netherlands     |11629       |
|Belgium         |5464        |
|Austria         |5043        |
|Switzerland     |3640        |
|United Kingdom  |3382        |
|Poland          |2139        |
|Czechia         |2034        |
|Italy           |2025        |
|Spain           |1651        |
|Portugal        |1336        |
|Sweden          |975         |
|Denmark         |905         |
|Serbia          |746         |
|Norway          |716         |
|Slovakia        |534         |
|Slovenia        |495         |
|Turkey          |485         |
|Greece          |412         |
|Lithuania       |351         |
|Latvia          |280         |
|Croatia         |254         |
|Estonia         |148         |
|Finland         |81          |
|Luxembourg      |68          |
|Andorra         |46          |
+----------------+------------+



## Câu 4: Phân tích số lượng đơn hàng nhóm theo năm, tháng đặt hàng (Hiển thị theo năm tăng dần, tháng giảm dần)

In [22]:
orders_df.select("Order_ID", "Order_Purchase_Timestamp")
orders_df.show(5, truncate=False)

+--------------------------------+--------------------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|Order_ID                        |Customer_Trx_ID                 |Order_Status|Order_Purchase_Timestamp|Order_Approved_At  |Order_Delivered_Carrier_Date|Order_Delivered_Customer_Date|Order_Estimated_Delivery_Date|
+--------------------------------+--------------------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b7cc49136f2d6af7|9ef432eb6251297304e76186b10a928d|delivered   |2023-10-02 10:56:00     |2023-10-02 11:07:00|2023-10-04 19:55:00         |2023-10-10 21:25:00          |2023-10-18 00:00:00          |
|53cdb2fc8bc7dce0b6741e2150273451|b0830fb4747a6c6d20dea0b8c802d7ef|delivered   |2024-07-24 20:41:00     |2024-07-26 03:24:00|2024-07-26 14:3

In [23]:
#Trích xuất year và month từ timestamp
extract_time = orders_df.select(
    "Order_ID",
    F.year(F.to_timestamp("Order_Purchase_Timestamp")).alias("year"),
    F.month(F.to_timestamp("Order_Purchase_Timestamp")).alias("month"),
)

#gom nhóm theo year và month
orders_time = (extract_time
    .groupBy("year", "month")
    .agg(F.countDistinct("Order_ID").alias("num_orders"))
    .orderBy(F.col("year").asc(), F.col("month").desc())
)
orders_time.show(50, truncate=False)

+----+-----+----------+
|year|month|num_orders|
+----+-----+----------+
|2022|12   |1         |
|2022|10   |324       |
|2022|9    |4         |
|2023|12   |5673      |
|2023|11   |7544      |
|2023|10   |4631      |
|2023|9    |4285      |
|2023|8    |4331      |
|2023|7    |4026      |
|2023|6    |3245      |
|2023|5    |3700      |
|2023|4    |2404      |
|2023|3    |2682      |
|2023|2    |1780      |
|2023|1    |800       |
|2024|10   |4         |
|2024|9    |16        |
|2024|8    |6512      |
|2024|7    |6292      |
|2024|6    |6167      |
|2024|5    |6873      |
|2024|4    |6939      |
|2024|3    |7211      |
|2024|2    |6728      |
|2024|1    |7269      |
+----+-----+----------+



## Câu 5: Thống kê điểm đánh giá trung bình, số lượng đánh giá theo từng mức (ví dụ: 1 đến 5).

In [24]:
from pyspark.sql import functions as F

reviews_processed = (
    reviews_df
    #Xử lí giá trị ngoại lai và null trong cột Review_Score
    .withColumn(
        "Review_Score_int",
        F.when(
            #Chỉ lấy giá trị [1,5]
            F.col("Review_Score").rlike(r"^\s*[1-5]\s*$"),
            #CHuyển string thành int
            F.col("Review_Score").cast("int")
        ).otherwise(F.lit(None).cast("int"))
    )
    .filter(F.col("Review_Score_int").isNotNull())
)

#tính avg
avg_score = reviews_processed.agg(F.avg("Review_Score_int").alias("Avg Review Score"))

count_level = (
    reviews_processed
    .groupBy("Review_Score_int")
    .agg(F.count("*").alias("num_reviews"))
    .orderBy("Review_Score_int")
)

avg_score.show()
count_level.show()

+------------------+
|  Avg Review Score|
+------------------+
|4.0864214950162765|
+------------------+

+----------------+-----------+
|Review_Score_int|num_reviews|
+----------------+-----------+
|               1|      11424|
|               2|       3151|
|               3|       8179|
|               4|      19141|
|               5|      57328|
+----------------+-----------+



## Câu 6: Tính doanh thu (giá sản phẩm + phí vận chuyển) trong năm 2024 và nhóm theo danh mục sản phẩm

In [25]:
from pyspark.sql import functions as F
#lấy ra số orders trong năm 2024
orders_2024 = (
    orders_df
    .withColumn(
        "purchase_ts",
        F.coalesce(
            F.to_timestamp("Order_Purchase_Timestamp", "yyyy-MM-dd HH:mm:ss"),        )
    )
    .filter(F.year("purchase_ts") == 2024)
    .select("Order_ID")
)

In [26]:
#join với orders_2024 để tính doanh thu sản phẩm
items_2024 = (
    items_df
    .join(orders_2024, on="Order_ID", how="inner")
    .withColumn("Price_d", F.col("Price").cast("double"))
    .withColumn("Freight_d", F.col("Freight_Value").cast("double"))
    .withColumn(
        "item_revenue",
        F.coalesce(F.col("Price_d"), F.lit(0.0)) + F.coalesce(F.col("Freight_d"), F.lit(0.0))
    )
)

In [27]:
#join với products_dt để gom nhóm theo category
revenue_by_category_2024 = (
    items_2024
    .join(products_df.select("Product_ID", "Product_Category_Name"), on="Product_ID", how="inner")
    .withColumn("Product_Category_Name", F.coalesce(F.col("Product_Category_Name"), F.lit("UNKNOWN")))
    .groupBy("Product_Category_Name")
    .agg(
        F.sum("item_revenue").alias("revenue_2024"),
        F.countDistinct("Order_ID").alias("num_orders"),
        F.count("*").alias("num_items")
    )
    .orderBy(F.col("revenue_2024").desc())
)

revenue_by_category_2024.show(50, truncate=False)

+---------------------------------------+------------------+----------+---------+
|Product_Category_Name                  |revenue_2024      |num_orders|num_items|
+---------------------------------------+------------------+----------+---------+
|Health_Beauty                          |885191.1199999986 |5402      |5951     |
|Watches_Gifts                          |771986.7500000006 |3493      |3703     |
|Bed_Bath_Table                         |650794.7000000007 |4909      |5884     |
|Sports_Leisure                         |621999.3399999999 |4058      |4527     |
|Computers_Accessories                  |594771.0399999988 |4053      |4708     |
|Housewares                             |491576.96000000037|3421      |4046     |
|Furniture_Decor                        |476466.1300000002 |3199      |4118     |
|Auto                                   |404210.5700000002 |2457      |2619     |
|Baby                                   |299052.5600000002 |1667      |1776     |
|Cool_Stuff     